**Bài thực hành lấy điểm lần 1: Phân lớp văn bản tiếng Anh:**

*   Họ tên: Đỗ Tấn Lực
*   Mã số sinh viên: 23520903
*   Lớp: CS221.P21



# 1. Cài đặt các thư viện.

In [ ]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter, defaultdict
from sklearn.preprocessing import LabelEncoder
import math
from sklearn.metrics import precision_score, recall_score, classification_report, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Tiếp theo, ta load dữ liệu văn bản tiếng Anh vào bằng thư viện Pandas.

In [ ]:
data = pd.read_csv('example.csv')
data.info()

Ta thống kê sơ bộ về ngữ liệu, về số lượng mẫu, datapoint mỗi class.

In [ ]:
cols= sns.color_palette("Reds", n_colors=2)
plt.figure(figsize=(10,6))
fig = sns.countplot(data=data, x='Class', hue="Class", palette= cols, legend=False)
fig.set_title("Thống kê số mẫu theo lớp", color="#000000")
fig.set_xlabel("Lớp")
fig.set_ylabel("Số mẫu")

# 2. Tiền xử lí dữ liệu.

### a. Tokenize

Tiếp theo, là tiền xử lý dữ liệu, ở đây là các văn bản tiếng Anh. Chúng ta sẽ thực hiện:

- Chuyển văn bản về chữ thường.
- Loại bỏ các kí tự không phải chữ cái.
- Tách từ.
- Dùng Stemming/Lemmatization chuyển văn bản về dạng gốc.

In [ ]:
def preprocess_text(text, use_lemmatizer=False):
    text = text.lower() # Chuyển thành chữ thường
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) # Loại bỏ các ký tự đặc biệt
    tokens = text.split() # Tokenize theo khoảng trắng
    stop_words = set(stopwords.words('english')) # Loại bỏ stopwords
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming/Lemmatization
    if use_lemmatizer:
        # WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    else:
        # PorterStemmer
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    return tokens

### b. Vectorize

Tiếp theo, ta xây dựng class TF-IDF để vectorize văn bản theo tần suất.

In [ ]:
class TF_IDF:
  def __init__(self):
    self.vocab = {} # Từ điển lưu cặp giá trị: token - index
    self.vocab_size = 0 # Kích thước từ điển
    self.idf = {} # Lưu giá trị IDF cho mỗi token

  def fit(self, documents):
    total_token = set() # Lưu trữ từ điển của documents
    doc_freq = defaultdict(int) # Lưu trữ số document chứa token
    for doc in documents:
      unique_token = set(doc)
      for token in unique_token:
        doc_freq[token] +=1
        total_token.add(token)

    self.vocab = {token: id for id, token in enumerate(sorted(total_token))}
    self.vocab_size = len(self.vocab)
    total_doc = len(documents)

    for token, frequence in doc_freq.items():
      self.idf[token] = math.log(total_doc +1 / frequence + 1) + 1 # Tính giá trị IDF của từng token

    return self

  def transform(self, documents):
    X = np.zeros((len(documents), self.vocab_size))
    for doc_id, doc in enumerate(documents):
      count_token = Counter(doc) # Đếm tần suất của token
      for token, frequence in count_token.items():
        if token in self.vocab:
          X[doc_id, self.vocab[token]] = frequence * self.idf[token] # Tính TFIDF cho từ điển

    return X

  def fit_transform(self, documents):
    self.fit(documents)
    return self.transform(documents)

# 3. Cài đặt các mô hình.

## a. Multinomial Naive Bayes

In [ ]:
class MultinomialNaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha # Smoothing
        self.class_prob = None # Xác suất tiên nghiệm P(y)
        self.feature_prob = None # Xác suất đặc trưng xuất hiện trong mỗi class P(x | y)
        self.classes = None
        self.n_features = None
        self.feature_count = None
        self.class_count = None

    def fit(self, X, y):
        if not isinstance(y, np.ndarray):
            y = np.array(y)

        # Get dimensions and unique classes
        n_samples, self.n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        self.class_count = np.zeros(n_classes)
        for i, c in enumerate(self.classes):
            self.class_count[i] = np.sum(y == c)
        self.class_prob = self.class_count / n_samples # Tính xác suất tiên nghiệm P(y)

        self.feature_count = np.zeros((n_classes, self.n_features)) # Lưu trữ số đặc trưng
        for i, c in enumerate(self.classes):
            X_c = X[y == c] # Lấy tất cả documents thuộc từng class
            self.feature_count[i] = X_c.sum(axis=0) # Tính tổng số đặc trưng theo từng class
        self.total_words_per_class = np.sum(self.feature_count, axis=1) # Tính tổng số từ trong mỗi class

        self.feature_prob = np.zeros((n_classes, self.n_features))
        for i in range(n_classes):
            self.feature_prob[i] = (self.feature_count[i] + self.alpha) / (self.total_words_per_class[i] + self.alpha * self.n_features) # Tính xác suất đặc trưng xuất hiện trong class - Laplace Smoothing

        return self

    def predict(self, X):
        return self.classes[np.argmax(self.log_likelihood(X), axis=1)] # Lấy class có xác suất cao nhất

    def log_likelihood(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        log_likelihood = np.zeros((n_samples, n_classes))

        for i, c in enumerate(self.classes):
            log_likelihood[:, i] = np.log(self.class_prob[i]) # Tính log của xác suất tiên nghiệm log(P(y))
            if isinstance(X, np.ndarray):
                feature_prob_c = self.feature_prob[i]
                log_probs = np.log(np.maximum(feature_prob_c, 1e-10)) # Thêm max để tránh log0
                log_likelihood[:, i] += np.dot(X, log_probs)
            else:
                for j in range(n_samples):
                    for feature_idx, count in zip(X[j].indices, X[j].data):
                        prob = self.feature_prob[i, feature_idx]
                        if prob > 0:
                            log_likelihood[j, i] += count * math.log(prob)

        return log_likelihood

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

## b. Bernouli Naive Bayes


In [ ]:
class BernoulliNaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.class_prob = None
        self.feature_prob = None
        self.n_feature = None
        self.classes = None
        self.class_count = None

    def fit(self, X, y):
        if not isinstance(y, np.ndarray):
            y = np.array(y)

        n_sample, self.n_feature = X.shape
        self.classes = np.unique(y)
        n_class = len(self.classes)

        self.class_count = np.zeros(n_class)
        self.class_prob = np.zeros(n_class)

        for i, c in enumerate(self.classes):
            self.class_count[i] = np.sum(y == c)
            self.class_prob[i] = self.class_count[i] / n_sample # Tính xác suất tiên nghiệm của mỗi lớp P(y)

        self.feature_prob = np.zeros((n_class, self.n_feature)) # Lưu trữ xác suất đặc trưng xuất hiện trong mỗi lớp P(xi|y)

        for i, c in enumerate(self.classes):
            X_c = X[y == c] # Lấy tất cả các mẫu thuộc lớp c
            n_samples_c = X_c.shape[0]
            feature_count = np.sum(X_c > 0, axis=0) # Đếm số mẫu trong lớp c mà đặc trưng xuất hiện, khi xi = 1

            self.feature_prob[i] = (feature_count + self.alpha) / (n_samples_c + 2 * self.alpha) # Laplace smoothing

        return self

    def predict(self, X):
        return self.classes[np.argmax(self._joint_log_likelihood(X), axis=1)]

    def log_likelihood(self, X):
        n_sample = X.shape[0]
        n_class = len(self.classes)
        log_likelihood = np.zeros((n_sample, n_class))
        X_bool = X > 0 # Biến đổi X thành dạng nhị phân

        for i in range(n_class):
            log_likelihood[:, i] = np.log(self.class_prob[i])
            feature_prob_i = self.feature_prob[i] # Tính log likelihood
            present_features = np.log(feature_prob_i) # Tính xác suất cho đặc trưng xuất hiện (xi = 1)
            absent_features = np.log(1 - feature_prob_i) # Tính xác suất cho đặc trưng không xuất hiện (xi = 0)
            # Tính tổng log likelihood cho các đặc trưng xuất hiện và không xuất hiện
            # Áp dụng công thức: Σ[x_i=1] log(P(x_i=1|y)) + Σ[x_i=0] log(P(x_i=0|y))
            log_likelihood[:, i] += np.sum(X_bool * present_features + (~X_bool) * absent_features, axis=1)

        return log_likelihood

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

## c. Logistic Regression

In [ ]:
class LogisticReg:
  def __init__(self, learning_rate=1e3, max_iter=1000):
    self.weights = None
    self.learning_rate = learning_rate
    self.max_iter = max_iter
    self.bias = 0

  def sigmoid(self, z):
    return 1 / (1 + np.exp(-np.clip(z, -100, 100))) # Tránh tràn số

  def fit(self, X, y):
    n_sample, n_feature = X.shape
    self.weights = np.zeros(n_feature)

    for i in range(self.max_iter):
      linear_model = np.dot(X, self.weights) + self.bias
      y_pred = self.sigmoid(linear_model)
      dw = (1 / n_sample) * np.dot(X.T, (y_pred - y)) # Tính gradient của trọng số W
      db = (1 / n_sample) * np.sum(y_pred - y) # Tính gradient của bias
      self.weights -= self.learning_rate * dw # Cập nhật trọng số
      self.bias -= self.learning_rate * db # Cập nhật bias

  def predict(self, X):
    return [1 if i > 0.5 else 0 for i in self.sigmoid(np.dot(X, self.weights) + self.bias)]

  def score(self, X, y):
    y_pred = self.predict(X)
    return accuracy_score(y, y_pred)


# 4. Cài đặt hàm run

In [ ]:
def process(TrainClass0, TrainClass1, Test):
  Trainclass0_pre = [preprocess_text(text) for text in TrainClass0] # Tiền xử lí cho Class 0
  Trainclass1_pre = [preprocess_text(text) for text in TrainClass1] # Tiền xử lí cho Class 1
  Test_pre = [preprocess_text(text) for text in Test] # Tiền xử lí cho tập Test

  vectorize = TF_IDF()
  X_train_pre = Trainclass0_pre + Trainclass1_pre # Tạo tập X_train
  y_train = [0] * len(Trainclass0_pre) + [1] * len(Trainclass1_pre) # Tạo y_train

  X_train = vectorize.fit_transform(X_train_pre)
  X_test = vectorize.transform(Test_pre)

  return X_train, X_test, y_train

In [ ]:
def run(TrainClass0, TrainClass1, Test):
  X_train, X_test, y_train = process(TrainClass0, TrainClass1, Test)
  #model = MultinomialNaiveBayes()
  model = BernoulliNaiveBayes()
  #model = LogisticReg()
  model.fit(X_train, y_train)
  return model.predict(X_test)

In [ ]:
label_encoder = LabelEncoder()
data["Class"] = label_encoder.fit_transform(data["Class"])
X = data["Content"]
y = data["Class"]
X_train, Test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
TrainClass0 = X_train[y==0]
TrainClass1 = X_train[y==1]


In [ ]:
print(run(TrainClass0, TrainClass1, Test))

[0 0 0 ... 0 0 1]
